<div style="width: 30%; float: right; margin: 10px; margin-right: 5%;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d3/FHNW_Logo.svg/2560px-FHNW_Logo.svg.png" width="500" style="float: left; filter: invert(50%);"/>
</div>

<h1 style="text-align: left; margin-top: 10px; float: left; width: 60%;">
    npr Mini-Challenge 2 <br> 
</h1>

<p style="clear: both; text-align: left;">
    Bearbeitet durch Si Ben Tran, Yannic Lais, Rami Tarabishi im HS 2023.<br>Bachelor of Science FHNW in Data Science.
</p>


# Aufgabe aus dem Space
Mini-Challenge 2-C (LE6):
Chatbot

You build a simple chatbot for hotel recommendation (here you can combine with npr and webscraping challenge) or extend the chatbot from the library (auxilio, please take contact with me), with rasa or dialogflow. You can also propose a use case. Important is the way intent are recognized and NER or variables are extracted. Delivery is a report on the use case, interactions and how intents are recognized and variables extracted. Also an error analysis on concrete conversations (confidence of models, explanation of predictions, etc.) should be undertaken. The submission can be done in a group of 2 or 3.


# 1 Libraries

In [1]:
%load_ext autoreload
%autoreload 2
from src.preprocessing import tokenize, stem, bag_of_words
from src.news_scraper import scrape_srf_links, get_data_for_topics
from src.json_concat import concat_json_files
from src.neuralmodel import NeuralNet 

import numpy as np
import json
from tqdm import tqdm 


import nltk
# nltk.download('punkt')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


One or more input files not found.


# 2 Preprocessor 

## 2.1 Test Tokenizer

In [2]:
word_list = ["believe", "believed", "believes", "believing", "believer"]
print(word_list)
stemmed_words = [stem(w) for w in word_list]
print(stemmed_words)

['believe', 'believed', 'believes', 'believing', 'believer']
['believ', 'believ', 'believ', 'believ', 'believ']


## 2.2 Test Stemmer

In [3]:
word_list = ["believe", "believed", "believes", "believing", "believer"]
print(word_list)
stemmed_words = [stem(w) for w in word_list]
print(stemmed_words)

['believe', 'believed', 'believes', 'believing', 'believer']
['believ', 'believ', 'believ', 'believ', 'believ']


## 2.3 Bag of Words

In [4]:
sentence = ["hello", "how", "are", "you"]
words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
bag = bag_of_words(sentence, words)
print(bag)

[0. 1. 0. 1. 0. 0. 0.]


# 3 Create and Load Data

In [5]:
# Scrape srf webpage
url = "https://www.srf.ch"
topic_list = scrape_srf_links(url)

# create data 
get_data_for_topics(topic_list)

Combined JSON file 'data/intents_news.json' created successfully for all topics.


In [6]:
with open('data/intents_news.json', 'r') as f:
    intents_news = json.load(f)
with open('data/intents-scratch.json', 'r') as f:
    intents = json.load(f)

# combine intents_news with intens
intents["intents"].extend(intents_news["intents"])
# export intents formmated as json
with open('data/intents.json', 'w') as outfile:
    json.dump(intents, outfile, indent=4)

In [7]:
all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents["intents"]:
    tag = intent["tag"]
    # add to tag list
    tags.append(tag)
    for pattern in intent["patterns"]:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

print(all_words)
print(tags)
print(xy)

['Hello', ',', 'my', 'name', 'is', 'Hi', ',', 'I', "'m", 'Hey', ',', 'I', 'go', 'by', 'I', 'am', 'Greetings', ',', 'call', 'me', 'You', 'can', 'call', 'me', 'They', 'call', 'me', 'My', 'name', "'s", 'I', "'d", 'like', 'to', 'introduce', 'myself', ',', 'I', "'m", 'Allow', 'me', 'to', 'introduce', 'myself', ',', 'I', 'am', 'I', 'go', 'by', 'the', 'name', 'People', 'know', 'me', 'as', 'Hi', 'Hey', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'Whats', 'up', 'How', 'are', 'you', 'doing', 'Howdy', 'Hi', 'there', 'Hola', 'Good', 'morning', 'Good', 'afternoon', 'Good', 'evening', 'It', 'is', 'nice', 'to', 'meet', 'you', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time', 'Have', 'a', 'nice', 'day', 'See', 'you', '!', 'Later', '!', 'Goodbye', '!', 'I', "'m", 'off', '!', 'That', "'s", 'all', 'for', 'now', '.', 'See', 'you', 'soon', '!', 'I', "'ve", 'got', 'to', 'get', 'going', '.', 'Bye', '!', 'It', 'was', 'n

# 3.1 Preprocessing Intent Data

In [8]:
# stem and lower each word and remove duplicates and sort
ignore_words = ["?", "!", ".", ","]
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

427 patterns
45 tags: ['arbeitsrecht', 'decodar-nossa-cultura', 'delivery', 'dialog', 'funny', 'fussball', 'gesellschaft', 'gesundheit', 'goodbye', 'greeting', 'international', 'introduction', 'items', 'jobs.srf.ch', 'kassensturz-espresso', 'kunst', 'literatur', 'live', 'mehr-sport', 'meteo-stories', 'musik', 'payments', 'radio', 'resultcenter', 'school', 'schweiz', 'ski-alpin', 'startseite', 'thanks', 'themen', 'tv', 'video', 'website-und-apps', 'wetter', 'wirtschaft', 'woerterbuch', 'www.radioswissclassic.ch', 'www.radioswissjazz.ch', 'www.radioswisspop.ch', 'www.rsi.ch', 'www.rtr.ch', 'www.rts.ch', 'www.srf.ch', 'www.srgssr.ch', 'www.swissinfo.ch']
167 unique stemmed words: ["'d", "'m", "'s", "'ve", 'a', 'about', 'accept', 'afternoon', 'all', 'allow', 'am', 'amex', 'ani', 'anyon', 'appl', 'arbeitsrecht', 'are', 'as', 'awesom', 'bitcoin', 'bunch', 'by', 'bye', 'call', 'can', 'card', 'cash', 'chat', 'convers', 'credit', 'crypto', 'cryptocurr', 'day', 'decodar-nossa-cultura', 'deliveri

## 3.2 Train and Test Data

In [9]:
X_train = []
y_train = []

for pattern_sentence, tag in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

print(X_train.shape)
print(X_train)
print(y_train.shape)
print(y_train)

(427, 167)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(427,)
[11 11 11 11 11 11 11 11 11 11 11 11  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8 28 28 28
 28 28 28 28 28 28 28 28 28 12 12 12 12 12 12 12 12 12 12 21 21 21 21 21
 21 21 21 21 21 21 21 21 21 21  2  2  2  2  2  2  2  4  4  4  4  4  5  5
  5  5  5  5  5  5  5 23 23 23 23 23 23 23 23 23 34 34 34 34 34 34 34 34
 34 43 43 43 43 43 43 43 43 43 31 31 31 31 31 31 31 31 31  3  3  3  3  3
  3  3  3  3 14 14 14 14 14 14 14 14 14 37 37 37 37 37 37 37 37 37 39 39
 39 39 39 39 39 39 39 30 30 30 30 30 30 30 30 30 24 24 24 24 24 24 24 24
 24 10 10 10 10 10 10 10 10 10 15 15 15 15 15 15 15 15 15 16 16 16 16 16
 16 16 16 16 20 20 20 20 20 20 20 20 20 19 19 19 19 19 19 19 19 19 38 38
 38 38 38 38 38 38 38 27 27 27 27 27 27 27 27 27 40 40 40 40 40 40 40 40
 40 13 13 13 13 13 13 1

## 3.3 Create DataLoader 

In [10]:
class ChatDataset(Dataset):
    def __init__(self, X_train, y_train):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [11]:
# define Batch size
batch_size = 2

dataset = ChatDataset(X_train=X_train, y_train=y_train)

print(dataset.n_samples)
print(dataset.x_data)
print(dataset.y_data)

# create DataLoader
train_loader = DataLoader(
    dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0
)

# get first batch
for first_batch in train_loader:
    break
# get first training batch
first_batch_inputs = first_batch[0]
first_batch_labels = first_batch[1]
print(first_batch_inputs)
print(first_batch_labels)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

427
[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[11 11 11 11 11 11 11 11 11 11 11 11  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8 28 28 28
 28 28 28 28 28 28 28 28 28 12 12 12 12 12 12 12 12 12 12 21 21 21 21 21
 21 21 21 21 21 21 21 21 21 21  2  2  2  2  2  2  2  4  4  4  4  4  5  5
  5  5  5  5  5  5  5 23 23 23 23 23 23 23 23 23 34 34 34 34 34 34 34 34
 34 43 43 43 43 43 43 43 43 43 31 31 31 31 31 31 31 31 31  3  3  3  3  3
  3  3  3  3 14 14 14 14 14 14 14 14 14 37 37 37 37 37 37 37 37 37 39 39
 39 39 39 39 39 39 39 30 30 30 30 30 30 30 30 30 24 24 24 24 24 24 24 24
 24 10 10 10 10 10 10 10 10 10 15 15 15 15 15 15 15 15 15 16 16 16 16 16
 16 16 16 16 20 20 20 20 20 20 20 20 20 19 19 19 19 19 19 19 19 19 38 38
 38 38 38 38 38 38 38 27 27 27 27 27 27 27 27 27 40 40 40 40 40 40 40 40
 40 13 13 13 13 13 13 13 13 13  6  6 

# 4 Model

Hier in diesem Abschnitt importieren wir unsere unterschiedlichen Modelle die wir für die Klassifizierung der Intents verwenden wollen.

# 4.1 Define Hyperparameters

In [12]:
# Hyper-parameters
batch_size = batch_size
num_epochs = 100
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 20
output_size = len(tags)
print(input_size, output_size)


167 45


## 4.2 Model Instanzieren

In [13]:
neural_net = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(neural_net.parameters(), lr=learning_rate)

# 5 Train Model

In [14]:
# Train the model
for epoch in tqdm(range(num_epochs), desc="Training Progress", unit="epoch"):
    # Wrap the train_loader with tqdm to create a progress bar
    for words, labels in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # Forward pass
        outputs = neural_net(words)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:  # Print loss after every 10 epochs
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.8f}")


print(f"final loss: {loss.item():.8f}")


Training Progress:  10%|█         | 10/100 [00:05<00:50,  1.80epoch/s]

Epoch [10/100], Loss: 0.07166445


Training Progress:  20%|██        | 20/100 [00:10<00:41,  1.91epoch/s]

Epoch [20/100], Loss: 0.02325162


Training Progress:  30%|███       | 30/100 [00:16<00:44,  1.57epoch/s]

...

Epoch [80/100], Loss: 0.00000310


Training Progress:  90%|█████████ | 90/100 [00:46<00:04,  2.15epoch/s]

Epoch [90/100], Loss: 0.00000083


Training Progress: 100%|██████████| 100/100 [00:51<00:00,  1.96epoch/s]

Epoch [100/100], Loss: 0.00000024
final loss: 0.00000024


## 5.1 Save Model

In [15]:
data = {
    "model_state": neural_net.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags,
}

FILE = "model/mlp_weights.pth"
torch.save(data, FILE)

print(f"training complete. file saved to {FILE}")

training complete. file saved to model/mlp_weights.pth


# 6 Load Model

In [16]:
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]

neural_net = NeuralNet(input_size, hidden_size, output_size).to(device)
neural_net.load_state_dict(model_state)
neural_net.eval()

bot_name = "Stupid Bounty Hunter Bot"
print("Let's chat! type 'quit' to exit")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = neural_net(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]

    if prob.item() > 0.75:
        for intent in intents["intents"]:
            if tag == intent["tag"]:
                print(f"{bot_name}: {np.random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! type 'quit' to exit


In [17]:
import torch
import numpy as np
import spacy

# Load the SpaCy NER model
nlp = spacy.load("en_core_web_sm")

# Function to extract persons from user input using SpaCy NER
def extract_persons(text):
    doc = nlp(text)
    persons = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
    return persons

# Your existing code for chatbot
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]

neural_net = NeuralNet(input_size, hidden_size, output_size).to(device)
neural_net.load_state_dict(model_state)
neural_net.eval()

bot_name = "Stupid Bounty Hunter Bot"
print("Let's chat! type 'quit' to exit")
while True:
    sentence = input("You: ")
    if sentence == "quit":
        break

    # Extract persons from the user input
    persons = extract_persons(sentence)

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = neural_net(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]

    if prob.item() > 0.75:
        for intent in intents["intents"]:
            if tag == intent["tag"]:
                # Add NER results to the response
                response = np.random.choice(intent['responses'])
                if persons:
                    response = response.replace("{persons}", ", ".join(persons))
                else:
                    response = response.replace("{persons}", "")
                print(f"{bot_name}: {response}")
    else:
        print(f"{bot_name}: I do not understand...")


Let's chat! type 'quit' to exit
Stupid Bounty Hunter Bot: Hi Rami, nice to meet you!
Stupid Bounty Hunter Bot: Hi Jelle, nice to meet you!
Stupid Bounty Hunter Bot: Hi, how can I help you Rami, Jelle?
Stupid Bounty Hunter Bot: Hi, what can I do for you today ?
Stupid Bounty Hunter Bot: Hi there, what can I do for you ? 
Stupid Bounty Hunter Bot: Hello, thanks for visiting 
Stupid Bounty Hunter Bot: Hello, thanks for visiting 
Stupid Bounty Hunter Bot: Hi, what can I do for you today ?
